In [1]:
# import os
# os.environ["ROM_DIR"] = "/net/csefiles/xzhanglab/mlobo6/miniconda3/envs/drl/lib/python3.10/site-packages/AutoROM/roms"

In [2]:
import gymnasium as gym
import ale_py
from ale_py import ALEInterface

In [ ]:
%%capture
# @title Imports
import torch
from torch.distributions import Categorical
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import random
import copy
import numpy as np

# code should work on either, faster on gpu
device = "cuda:2" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# random seeds for reproducability
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 2154436), started 2 days, 1:04:01 ago. (Use '!kill 2154436' to kill it.)

In [ ]:
import gymnasium as gym
import ale_py
from ale_py import ALEInterface

import torch
from torch.distributions import Categorical
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import random
import copy
import numpy as np
from datetime import datetime

import cv2
import matplotlib.pyplot as plt
import imageio
import tqdm

from sklearn.manifold import TSNE
from torchvision.transforms import ToTensor, Resize
from PIL import Image
import gc

import os
from gymnasium.wrappers import RecordVideo

from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA
 

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
now = datetime.now()
timestamp = now.strftime("%d_%H%M%S")
writer = SummaryWriter(log_dir=f'runs/SAC-Discrete-{timestamp}')

In [ ]:
ale = ALEInterface()
gym.register_envs(ale_py)

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


In [ ]:
def cleanup_memory():
    gc.collect()  # Clean up CPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()            # Release cached GPU memory (PyTorch only)
        torch.cuda.ipc_collect()            # Clean up interprocess memory (multi-GPU safe)


In [ ]:
import warnings

In [ ]:
# store experience in a replay buffer, sample from it 
class ReplayBuffer:
    def __init__(self, capacity=100_000):
        self.buffer = deque(maxlen=capacity)
        self.batch_size = 256 # 512

    def store(self, state, skill, action, reward, next_state, done):
        transitions = list(zip(state, skill, action, reward, next_state, 1 - torch.Tensor(done)))
        self.buffer.extend(transitions)

    def sample(self):
        if len(self.buffer) < self.batch_size:
            batch = random.choices(self.buffer, k=self.batch_size)
            warnings.warn(f"Requested batch size {self.batch_size} is larger than buffer size \
                 {len(self.buffer)}. Sampling with replacement.", category=UserWarning)

        else:
            batch = random.sample(self.buffer, self.batch_size)
        
        states, skills, actions, rewards, next_states, not_dones = zip(*batch)

        # stack and move to device
        states      = torch.stack(states,     dim=0).to(device)
        skills      = torch.stack(skills,     dim=0).to(device)
        actions     = torch.stack(actions,    dim=0).to(device)
        rewards     = torch.stack(rewards,    dim=0).to(device)
        next_states = torch.stack(next_states,dim=0).to(device)
        not_done    = torch.stack(not_dones,  dim=0).to(device).unsqueeze(-1)  # [B,1] # .

        return states, skills, actions, rewards, next_states, not_done

        # return [torch.stack(e).to(device) for e in zip(*batch)]  # state, skill, action, reward, next_state, not_done
    
    def sample_by_skill(self, skill_id, num_samples=128): ## THIS METHOD ASSUMES ONE HOT ENCODED SKILLS
        filtered = [transition for transition in self.buffer
                    if torch.argmax(transition[1]).item() == skill_id]

        if len(filtered) < num_samples:
            # batch = filtered
            batch = random.choices(filtered, k=num_samples)
            warnings.warn(f"Not enough samples for skill {skill_id}. Sampling with replacement.", category=UserWarning)
        else:
            batch = random.sample(filtered, num_samples)

        states, skills, actions, rewards, next_states, not_dones = zip(*batch)
        states      = torch.stack(states,     dim=0).to(device)
        skills      = torch.stack(skills,     dim=0).to(device)
        actions     = torch.stack(actions,    dim=0).to(device)
        rewards     = torch.stack(rewards,    dim=0).to(device)
        next_states = torch.stack(next_states,dim=0).to(device)
        not_done    = torch.stack(not_dones,  dim=0).to(device).unsqueeze(-1) # .to(self.device)

        return states, skills, actions, rewards, next_states, not_done

        # return [torch.stack(e).to(device) for e in zip(*batch)]


    def __len__(self):
        return len(self.buffer)


In [ ]:
# # to encode input pacman images 
# class CNNEncoder(nn.Module):
#     def __init__(self, out_dim):
#         super().__init__()
#         self.conv = nn.Sequential(
#             nn.Conv2d(3, 32, kernel_size=8, stride=4),  # (3, 210, 160) → (32, 52, 39)
#             nn.ReLU(),
#             nn.Conv2d(32, 64, kernel_size=4, stride=2), # → (64, 25, 18)
#             nn.ReLU(),
#             nn.Conv2d(64, 64, kernel_size=3, stride=1), # → (64, 23, 16)
#             nn.ReLU()
#         ).to(device)
#         self.fc = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(64 * 22 * 16, out_dim),
#             nn.ReLU()
#         ).to(device)

#     def forward(self, x):
#         return self.fc(self.conv(x / 255.0))




In [ ]:
# class DRL:
#     def __init__(self, buffer_size = 10000):
#         self.n_envs = 32
#         self.episodes_per_epoch = 8 # 400 
#         # self.n_steps =  512

#         self.envs = gym.vector.SyncVectorEnv(
#             [lambda: gym.make('ALE/MsPacman-v5') for _ in range(self.n_envs)])

#         self.replay_buffer = ReplayBuffer(capacity=buffer_size)

#     def rollout(self, agent, epoch, n_skill, encoder): # one episode 
#         """Collect experience and store it in the replay buffer"""
#         encoder = encoder.to(device)
        
#         # obs, _ = self.envs.reset()
#         # obs = torch.tensor(obs, dtype=torch.float32, device=device).permute(0, 3, 1, 2)# .to(device)
#         # enc_obs = encoder(obs)

#         obs_np, _ = self.envs.reset()
#         obs       = torch.tensor(obs_np, dtype=torch.float32).permute(0,3,1,2).to(device)
#         with torch.no_grad():
#             enc_obs = encoder(obs)
#         # env_skills = F.normalize(torch.randn(self.n_envs, n_skill, device=device), dim=1)  # shape: (n_envs, n_skill)
#         # total_rewards = torch.zeros(self.n_envs, device=device)

#         skill_ids  = torch.randint(0, n_skill, (self.n_envs,), device=device)
#         env_skills = F.one_hot(skill_ids, n_skill).float().to(device)
        
#         # tracking
#         episodes_done   = 0
#         episode_returns = torch.zeros(self.n_envs, device=device)
#         all_episode_returns = []
        
#         # for step_num in range(self.n_steps): # steps in the trajectory 
#         #     with torch.no_grad():
#         #         actions = agent.get_action(enc_obs, env_skills)

#         #     next_obs, rewards, dones, truncateds, _ = self.envs.step(actions.cpu().numpy())
#         #     next_obs = torch.tensor(next_obs, dtype=torch.float32, device=device).permute(0, 3, 1, 2)
#         #     rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
#         #     not_done = ~(dones | truncateds)

#         #     # Store the transitions
#         #     self.replay_buffer.store(obs, env_skills, actions, rewards, next_obs, not_done)
#         #     obs = next_obs
#         #     with torch.no_grad():
#         #         enc_obs = encoder(obs)
#         #     total_rewards += rewards

#         #     # Resample skills only for environments where episode ended
#         #     done_mask = dones | truncateds
#         #     if done_mask.any() or step_num % 1000 == 0: ## RESAMPLING SKILLS QUITE A LOT :)
#         #         new_skills = F.normalize(torch.randn(done_mask.sum(), n_skill, device=device), dim=1)
#         #         env_skills[done_mask] = new_skills
            
#         #     # Resample skill at every iteration
#         #     # new_skills = F.normalize(torch.randn(self.n_envs, n_skill, device=device), dim=1)
#         #     # env_skills = new_skills
        
#         while episodes_done < self.episodes_per_epoch:
#             # select and step
#             with torch.no_grad():
#                 actions = agent.get_action(enc_obs, env_skills)
#             next_obs, rewards, dones, truncs, _ = self.envs.step(actions.cpu().numpy())
#             next_obs = torch.tensor(next_obs, dtype=torch.float32, device=device).permute(0,3,1,2)
#             rewards  = torch.tensor(rewards, dtype=torch.float32, device=device)
#             not_done = ~(dones | truncs)

#             # store transitions
#             self.replay_buffer.store(obs, env_skills, actions, rewards, next_obs, not_done)

#             # accumulate returns
#             episode_returns += rewards

#             # detect finished envs
#             done_mask = dones | truncs

#             # done_mask
#             if done_mask.any():
#                 # record returns for each finished env
#                 finished_idxs = torch.tensor(done_mask).to(bool).nonzero(as_tuple=True)[0]
#                 for idx in finished_idxs:
#                     all_episode_returns.append(episode_returns[idx].item())
#                 episodes_done += finished_idxs.size(0)

#                 # reset those envs
#                 # (SyncVectorEnv only supports full reset, so we reset all and copy back)
#                 new_obs, _ = self.envs.reset()
#                 new_obs = torch.tensor(new_obs, dtype=torch.float32, device=device).permute(0,3,1,2)
#                 obs[done_mask] = new_obs[done_mask]

#                 # sample new skills for those envs
#                 # new_ids = torch.randint(0, n_skill, (finished_idxs.size(0),), device=self.device)
#                 # skill_ids[done_mask] = new_ids
#                 # env_skills = F.one_hot(skill_ids, n_skill).float()

#                 # zero their return counters
#                 episode_returns[done_mask] = 0.0
#                 # re-encode only reset entries
#                 with torch.no_grad():
#                     enc_obs = encoder(obs)
#                 del next_obs, rewards, dones, truncs, not_done, actions
#                 torch.cuda.empty_cache()

#             else:
#                 obs    = next_obs
#                 with torch.no_grad():    
#                     enc_obs = encoder(obs)
#                 del next_obs, rewards, dones, truncs, not_done, actions
#                 torch.cuda.empty_cache()
        
#         avg_ret = sum(all_episode_returns) / len(all_episode_returns)
#         writer.add_scalar("stats/Rewards", avg_ret, epoch)
#         # writer.add_scalar("stats/Rewards", total_rewards.mean().item() / self.n_steps, i)

In [ ]:
# class DRL:
    
#     def rollout(self, agent, epoch, n_skill, encoder, episodes_per_epoch=4):
#         """
#         Collect exactly `episodes_per_epoch` full episodes (across the n_envs envs),
#         store transitions in the shared replay buffer, and log avg return.
#         """
#         encoder = encoder.to(self.device)
#         # initial reset
#         obs, _ = self.envs.reset()
#         obs = torch.tensor(obs, dtype=torch.float32, device=self.device).permute(0,3,1,2)
#         enc_obs = encoder(obs)

#         # one skill per env for current episode
#         skill_ids  = torch.randint(0, n_skill, (self.n_envs,), device=self.device)
#         env_skills = F.one_hot(skill_ids, n_skill).float()

#         # tracking
#         episodes_done   = 0
#         episode_returns = torch.zeros(self.n_envs, device=self.device)
#         all_episode_returns = []

#         while episodes_done < episodes_per_epoch:
#             # select and step
#             with torch.no_grad():
#                 actions = agent.get_action(enc_obs, env_skills)
#             next_obs, rewards, dones, truncs, _ = self.envs.step(actions.cpu().numpy())
#             next_obs = torch.tensor(next_obs, dtype=torch.float32, device=self.device).permute(0,3,1,2)
#             rewards  = torch.tensor(rewards, dtype=torch.float32, device=self.device)
#             not_done = ~(dones | truncs)

#             # store transitions
#             self.replay_buffer.store(obs, env_skills, actions, rewards, next_obs, not_done)

#             # accumulate returns
#             episode_returns += rewards

#             # detect finished envs
#             done_mask = dones | truncs
#             if done_mask.any():
#                 # record returns for each finished env
#                 finished_idxs = done_mask.nonzero(as_tuple=True)[0]
#                 for idx in finished_idxs:
#                     all_episode_returns.append(episode_returns[idx].item())
#                 episodes_done += finished_idxs.size(0)

#                 # reset those envs
#                 # (SyncVectorEnv only supports full reset, so we reset all and copy back)
#                 new_obs, _ = self.envs.reset()
#                 new_obs = torch.tensor(new_obs, dtype=torch.float32, device=self.device).permute(0,3,1,2)
#                 obs[done_mask] = new_obs[done_mask]

#                 # sample new skills for those envs
#                 new_ids = torch.randint(0, n_skill, (finished_idxs.size(0),), device=self.device)
#                 skill_ids[done_mask] = new_ids
#                 env_skills = F.one_hot(skill_ids, n_skill).float()

#                 # zero their return counters
#                 episode_returns[done_mask] = 0.0
#                 # re-encode only reset entries
#                 enc_obs = encoder(obs)

#             else:
#                 obs    = next_obs
#                 enc_obs = encoder(obs)

#         # log average return over the episodes we just finished
#         avg_return = sum(all_episode_returns) / len(all_episode_returns)
#         writer.add_scalar("stats/episode_return", avg_return, epoch)


In [ ]:
class DRL:
    def __init__(self, buffer_size = 100000):
        self.n_envs = 32
        self.n_steps = 512

        self.envs = gym.vector.SyncVectorEnv(
            [lambda: gym.make('ALE/MsPacman-v5') for _ in range(self.n_envs)])

        self.replay_buffer = ReplayBuffer(capacity=buffer_size)

    def rollout(self, agent, i, n_skill, encoder):
        """Collect experience and store it in the replay buffer"""
        encoder = encoder.to(device)
        obs, _ = self.envs.reset()
        obs = torch.tensor(obs, dtype=torch.float32, device=device).permute(0, 3, 1, 2)# .to(device)
        enc_obs = encoder(obs)

        env_skills = F.normalize(torch.randn(self.n_envs, n_skill, device=device), dim=1)  # shape: (n_envs, n_skill)
        total_rewards = torch.zeros(self.n_envs, device=device)


        for step_num in range(self.n_steps):
            with torch.no_grad():
                actions = agent.get_action(enc_obs, env_skills)

            next_obs, rewards, dones, truncateds, _ = self.envs.step(actions.cpu().numpy())
            next_obs = torch.tensor(next_obs, dtype=torch.float32, device=device).permute(0, 3, 1, 2)
            rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
            not_done = ~(dones | truncateds)

            # Store the transitions
            self.replay_buffer.store(obs, env_skills, actions, rewards, next_obs, not_done)
            obs = next_obs
            with torch.no_grad():
                enc_obs = encoder(obs)
            total_rewards += rewards

            # Resample skills only for environments where episode ended
            done_mask = dones | truncateds
            if done_mask.any() or step_num % 50 == 0: ## RESAMPLING SKILLS QUITE A LOT :)
                new_skills = F.normalize(torch.randn(done_mask.sum(), n_skill, device=device), dim=1)
                env_skills[done_mask] = new_skills


        writer.add_scalar("stats/Rewards", total_rewards.mean().item() / self.n_steps, i)

In [ ]:
def encode_state(state, encoder): # takes state and passes through CNNEncoder 
    if state.dim() == 4 and state.shape[1] == 3:
        features = encoder(state)
    elif state.dim() == 3 and state.shape[0] == 3:
        state = state.unsqueeze(0)
        features = encoder(state)
    elif state.dim() == 2:
        features = state
    else:
        raise ValueError(f'found a state with shape: {state.shape} in get_latent_representationss')
    
    return features

In [ ]:
class SkillPolicy: # implements discrete SAC 
    def __init__(self, n_obs, n_skills, n_actions, representation, tau=0.005, lr=1e-4, gamma = 0.99, automatic_entropy_tuning=True, par_alpha=0.2, target_entropy=None):
        
        
        self.alpha = par_alpha # 1.5
        self.n_actions = n_actions
        self.encoder = representation.encoder
        self.representation = representation
        self.tau = tau
        self.gamma = gamma

        # Q1Network
        self.q1_net = nn.Sequential(
            nn.Linear(n_obs + n_skills, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.n_actions)
        ).to(device)
        
        # Q2Network
        self.q2_net = copy.deepcopy(self.q1_net).to(device)

        # Policy Network
        self.policy = nn.Sequential(
            nn.Linear(n_obs + n_skills, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.n_actions)
        ).to(device)

        # Target Q1 Netowrk 
        self.q1_target_net = copy.deepcopy(self.q1_net).to(device)
        
        # Target Q2 network
        self.q2_target_net = copy.deepcopy(self.q1_net).to(device)

        # self.q1_target_net.load_state_dict(self.q1_net.state_dict()) # maybe faster
        # self.q2_target_net.load_state_dict(self.q2_net.state_dict())

        # Single Q optimizer 
        self.q_optimizer = Adam(
            list(self.q1_net.parameters()) + list(self.q2_net.parameters()), lr=lr
        )

        # Policy Optimizer 
        self.policy_optimizer = Adam(self.policy.parameters(), lr=lr)
        
        
        # self.gamma = gamma

        ###########   add temperature  ############
        self.automatic_entropy_tuning = automatic_entropy_tuning
        if automatic_entropy_tuning:
            # target entropy = −|A|
            self.target_entropy = -self.n_actions if target_entropy is None else target_entropy
            self.log_alpha = torch.zeros(1, requires_grad=True, device=device)
            self.alpha_optimizer = Adam([self.log_alpha], lr=lr)
        else:
            self.alpha = par_alpha
        ################################################





    ############ Selection of the action  #############
    def get_policy_distribution(self, states, skills):
        states = encode_state(states, self.encoder)
        inputs = torch.cat([states, skills], dim=-1)
        logits = self.policy(inputs)
        return Categorical(logits=logits)

    def get_action(self, states, skills, eval=False):
        dist = self.get_policy_distribution(states, skills)
        if eval:
            return torch.argmax(dist.probs, dim=-1)
        return dist.sample()
    ###################################################


    # def get_entropy(self, states, skills):
    #     dist = self.get_policy_distribution(states, skills)
    #     return dist.entropy()


    def get_q_loss(self, states, actions, rewards, next_states, not_dones, skills):

        # ext_reward = rewards.to(device)

        with torch.no_grad():
            phi = self.representation.get_latent_representation(states)  # phi(s) 
            phi_next = self.representation.get_latent_representation(next_states) # phi (s')

            delta_phi = phi - phi_next
            # skills = F.normalize(skills, p=2, dim=1)

            intrinsic_reward = torch.einsum('bi,bi->b', delta_phi, skills).unsqueeze(-1)
            intrinsic_reward = torch.clamp(intrinsic_reward, -10, 10)
            # add extrinsic reward ??
            
            ############## target Q value #######################
            next_states = encode_state(next_states, self.encoder)
            next_input = torch.cat([next_states, skills], dim=-1)
            next_q1 = self.q1_target_net(next_input)
            next_q2 = self.q2_target_net(next_input)
            next_q = torch.min(next_q1, next_q2)

            
            next_pi = self.get_policy_distribution(next_states, skills)
            log_probs = next_pi.logits.log_softmax(dim=-1)
            # next_entropy = next_pi.entropy().unsqueeze(-1)
            # next_entropy = next_pi.logits.log_softmax(dim=-1)

            if self.automatic_entropy_tuning:
                alpha = self.log_alpha.exp()
            else:
                alpha = self.alpha

            next_q_val = (next_pi.probs * (next_q - alpha * log_probs)).sum(dim=-1, keepdim=True)
            q_target = intrinsic_reward + (self.gamma * not_dones * next_q_val) # + ext_reward # added extrinisic reward, may not make sense
            #####################################################

        ############  Q losses ##################
        states = encode_state(states, self.encoder)
        current_inputs = torch.cat([states, skills], dim=-1)
        q1 = self.q1_net(current_inputs).gather(1, actions.long().unsqueeze(-1))
        q2 = self.q2_net(current_inputs).gather(1, actions.long().unsqueeze(-1))

        loss = F.mse_loss(q1, q_target) + F.mse_loss(q2, q_target)
        #######################################
        return loss
        

    #################### policy loss ##################
    def get_policy_loss(self, states, skills):
        dist = self.get_policy_distribution(states, skills)
        probs = dist.probs
        log_probs = dist.logits.log_softmax(dim=-1)

        states = encode_state(states, self.encoder)

        inputs = torch.cat([states, skills], dim=-1)
        q1 = self.q1_net(inputs)
        q2 = self.q2_net(inputs)
        q = torch.min(q1, q2)

        # added alpha 
        if self.automatic_entropy_tuning:
            alpha = self.log_alpha.exp()
        else:
            alpha = self.alpha


        policy_loss = -(probs * (q - (alpha * log_probs))).sum(dim=1).mean()
        return policy_loss
    #####################################################

    ############### entropy loss ###################
    def get_entropy_loss(self, states, skills):
        dist = self.get_policy_distribution(states, skills)
        # dist.entropy()
        probs = dist.probs
        log_probs = dist.logits.log_softmax(dim=-1)

        if self.automatic_entropy_tuning:
            alpha_loss = -(self.log_alpha * (log_probs + self.target_entropy).detach()).mean()
        else:
            alpha_loss = 0
        return alpha_loss
    # self.alpha_opt.zero_grad()
    #         alpha_loss.backward()
    #         self.alpha_opt.step()
    ###################################################    


    def update(self, replay_buffer, i):

      for _ in range(1): # i dont think its good idea to set this too high ?
        states, skills, actions, rewards, next_states, not_dones = replay_buffer.sample()
        # Compute Q-loss
        q_loss = self.get_q_loss(states, actions, rewards, next_states, not_dones, skills)
        self.q_optimizer.zero_grad()
        q_loss.backward()
        self.q_optimizer.step()

        # Update the policy
        policy_loss = self.get_policy_loss(states, skills)
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()

      # Soft update the target networks using Polyak averaging
      self.soft_update(self.q1_net, self.q1_target_net)
      self.soft_update(self.q2_net, self.q2_target_net)
      # entropy = self.get_entropy(states, skills).mean().item()

      alpha_loss = self.get_entropy_loss(states, skills)
      self.alpha_optimizer.zero_grad()
      alpha_loss.backward()
      self.alpha_optimizer.step()

      writer.add_scalar("loss/entropy_loss", alpha_loss, i)

      writer.add_scalar("loss/q_loss", q_loss.item(), i)
      writer.add_scalar("loss/ - policy loss", -policy_loss.item(), i)


    def soft_update(self, source, target):
        """Soft update the target network"""
        for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(self.tau * param.data + (1.0 - self.tau) * target_param.data)


In [ ]:
# import torch
# import torch.nn as nn

# class CNNEncoder(nn.Module):
#     """
#     CNN encoder loosely based on LeCun et al. (1989) but adapted for 3×210×160 inputs
#     and with two 1024‑unit hidden layers in the fully‑connected head.
    
#     Input:  (B, 3, 210, 160)
#     Output: (B, out_dim)
#     """
#     def __init__(self, out_dim: int):
#         super().__init__()
#         self.conv_net = nn.Sequential(
#             # C1: conv 3→32, 5×5, pad=2 → (32, 210, 160)
#             nn.Conv2d(3, 32, kernel_size=5, padding=2),
#             nn.Tanh(),
#             # S2: avg pool 2×2       → (32, 105,  80)
#             nn.AvgPool2d(kernel_size=2, stride=2),
            
#             # C3: conv 32→64, 5×5, pad=2 → (64, 105,  80)
#             nn.Conv2d(32, 64, kernel_size=5, padding=2),
#             nn.Tanh(),
#             # S4: avg pool 2×2       → (64,  52,  40)
#             nn.AvgPool2d(kernel_size=2, stride=2),
            
#             # C5: conv 64→128, 5×5, pad=2 → (128,  52,  40)
#             nn.Conv2d(64, 128, kernel_size=5, padding=2),
#             nn.Tanh(),
#             # S6: avg pool 2×2       → (128,  26,  20)
#             nn.AvgPool2d(kernel_size=2, stride=2),
            
#             nn.Flatten(),  # → (B, 128 * 26 * 20)
#         )
        
#         # Compute flattened size dynamically
#         with torch.no_grad():
#             dummy = torch.zeros(1, 3, 210, 160)
#             flat_size = self.conv_net(dummy).shape[1]  # should be 128*26*20 = 66560
        
#         # Fully‑connected head with 2 hidden layers of 1024 units each
#         self.fc_head = nn.Sequential(
#             nn.Linear(flat_size, 1024),
#             nn.Tanh(),
#             nn.Linear(1024, 1024),
#             nn.Tanh(),
#             nn.Linear(1024, out_dim)
#         )
        
#         # Initialize weights as in LeNet (Xavier for tanh)
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#                 nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('tanh'))
#                 if m.bias is not None:
#                     nn.init.zeros_(m.bias)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         x = self.conv_net(x/255.0)    # conv → flatten
#         return self.fc_head(x)  # two 1024‑unit hidden layers → out_dim


In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4),  # (3, 210, 160) → (32, 52, 39)
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), # → (64, 25, 18)
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), # → (64, 23, 16)
            nn.ReLU()
        ).to(device)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 16, out_dim),
            nn.ReLU()
        ).to(device)

    def forward(self, x):
        return self.fc(self.conv(x / 255.0))

In [ ]:

class RepresentationFunction(nn.Module): # Meat of METRA
    def __init__(self, n_obs, n_skill, lr=1e-4):
        super().__init__()

        # Encode input state 
        self.encoder = CNNEncoder(out_dim=256) # output of the CNN

        # phi network - n_obs is hardcoded to 256 in main
        self.representation_func = nn.Sequential(
            nn.Linear(n_obs, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, n_skill)
        ).to(device)

        # joint optimization over encoder and phi
        self.optimizer = Adam(list(self.parameters()) + list(self.encoder.parameters()), lr=1e-4)
        
        # lagrange mult
        self.lambda_param = nn.Parameter(torch.tensor(30.0, requires_grad=True, device=device))
        
        # lambda optimizer 
        self.lambda_optimizer = Adam([self.lambda_param], lr=1e-4)
        
        # threshold for metra constraint 
        self.epsilon = 0.001 # 1.0


    def get_latent_representation(self, state, normalize=False):
        features = encode_state(state, self.encoder) # Runs CNNEncoder 
        x = self.representation_func(features) # phi output 
        if normalize:
            return F.normalize(x, p=2, dim=1)
        return x


    def update(self, replay_buffer, i):
        """
        Updates the representation function φ and the Lagrange multiplier λ
        based on skill-consistency and distance constraints.
        """
        for _ in range(50): # 10

          state, skill, action, reward, next_state, not_done  = replay_buffer.sample()

          current_representations = self.get_latent_representation(state)    # φ(s)
          next_representations = self.get_latent_representation(next_state)  # φ(s')

          # Consistency loss term
          consistency_term = torch.einsum('bi,bi->b', (next_representations - current_representations), skill)

          # Distance penalty term (to enforce norm constraints)
          diff_norm_squared = torch.norm(current_representations - next_representations, dim=1) ** 2
          penalty_term = torch.minimum(
              torch.tensor(self.epsilon, device=diff_norm_squared.device),
              (1.0 - diff_norm_squared).clone()
          )

          # Representation loss
          representation_loss = -(consistency_term + self.lambda_param * penalty_term).mean()

          # Lambda loss (only on penalty term)
          lambda_loss = (self.lambda_param * penalty_term.detach()).mean()

          # Backprop: update \phi
          self.optimizer.zero_grad()
          representation_loss.backward(retain_graph=True)
          self.optimizer.step()

          # Backprop: update \lambda
          self.lambda_optimizer.zero_grad()
          lambda_loss.backward()
          self.lambda_optimizer.step()
          with torch.no_grad():
                self.lambda_param.clamp_(min=0.0)

        writer.add_scalar("loss/representation_loss", representation_loss.item(), i)
        writer.add_scalar("loss/  lambda_loss", lambda_loss.item(), i)





In [ ]:

# def compute_heatmap(frames, save_path):
#     heatmap = None
#     for frame in frames:
#         gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
#         # naive thresholding to isolate the player
#         _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
#         if heatmap is None:
#             heatmap = np.zeros_like(thresh, dtype=np.float32)
#         heatmap += thresh.astype(np.float32)

#     # Normalize and save heatmap
#     plt.figure(figsize=(6, 6))
#     plt.imshow(heatmap, cmap='hot', interpolation='nearest')
#     plt.axis('off')
#     plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

def compute_coverage(heatmap: np.ndarray) -> float:
    """
    Given a 2D heatmap of visit counts, returns the fraction of pixels
    that were ever visited (i.e. count>0) vs. the total area.
    """
    # How many pixels were ever nonzero?
    covered = np.count_nonzero(heatmap)
    total   = heatmap.size
    return covered / total

def compute_pacman_heatmap(
    frames,
    save_path=None,
    yellow_lower=(20, 100, 100),
    yellow_upper=(40, 255, 255),
    morph_kernel_size=3,
    blur_ksize=(21,21),
    overlay_alpha=0.6
):
    """
    Build a heatmap of Pac‑Man’s positions over a sequence of RGB frames.
    
    Args:
      frames: list of (H,W,3) uint8 RGB arrays.
      save_path: where to write the PNG.
      yellow_lower/upper: HSV thresholds for Pac‑Man’s color.
      morph_kernel_size: size for opening to remove noise.
      blur_ksize: Gaussian blur kernel to smooth final heatmap.
      overlay_alpha: how strongly to blend heatmap on top of the first frame.
    
    Returns:
      heatmap: float32 array of shape (H,W) with counts of visits.
      coverage: fraction of pixels ever visited.
    """
    # prepare  
    H, W, _ = frames[0].shape
    visit_map = np.zeros((H, W), dtype=np.float32)

    # set up morphological kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_kernel_size,)*2)

    for frame in frames:
        # 1) HSV color‐segment Pac‑Man
        hsv  = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(hsv, np.array(yellow_lower), np.array(yellow_upper))

        # 2) clean small speckles
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # 3) accumulate visit_map (binary) so repeated visits don’t accumulate linearly
        visit_map += (mask > 0).astype(np.float32)

    # 4) coverage: fraction of pixels ever visited
    coverage = np.count_nonzero(visit_map) / visit_map.size

    # 5) smooth for nicer appearance
    smooth_map = cv2.GaussianBlur(visit_map, blur_ksize, sigmaX=0)

    # 6) normalize to [0,1]
    norm_map = smooth_map / (smooth_map.max() + 1e-8)

    # 7) overlay on first frame
    background = frames[0].astype(np.float32)/255.0
    heatmap_rgb = plt.get_cmap('hot')(norm_map)[:,:,:3]  # H×W×3
    overlay = (1-overlay_alpha)*background + overlay_alpha*heatmap_rgb

    # 8) save
    if save_path is not None:
        plt.imsave(save_path, np.clip(overlay, 0, 1))

    return visit_map, coverage


def visualize_representation(replay_buffer, representation, global_step, n_samples=1000, folder='.', n_skills=10):
    skill_data = []
    for skill_id in range(n_skills):
        samples = replay_buffer.sample_by_skill(skill_id, num_samples=max(int(n_samples/10), 100))
        skill_data.append(samples)

    # Combine all samples
    states = torch.cat([s[0] for s in skill_data], dim=0)
    skills = torch.cat([s[1] for s in skill_data], dim=0)

    with torch.no_grad():
        skill_ids = torch.argmax(skills, dim=1)  # assume Gaussian skills aren't passed here!

        phis = representation.get_latent_representation(states).cpu().numpy()
        skill_ids_np = skill_ids.cpu().numpy()

    tsne = TSNE(n_components=2, perplexity=20)
    tsne_result = tsne.fit_transform(phis)


    plt.figure(figsize=(8, 6))
    for skill_id in range(n_skills):
        idx = skill_ids_np == skill_id
        if np.sum(idx) == 0:
            continue
        plt.scatter(tsne_result[idx, 0], tsne_result[idx, 1], label=f'Skill {skill_id}', alpha=0.6)
    plt.legend()
    plt.title('t-SNE of Representation φ(s)')
    plt.savefig(f"{folder}/representation_tsne.png")
    plt.close()

    if len(np.unique(skill_ids_np)) > 1:
        sil_score = silhouette_score(phis, skill_ids_np)
        writer.add_scalar("representation/silhouette_score", sil_score, global_step)


    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(phis)

    # Plot
    plt.figure(figsize=(8, 6))
    for skill_id in range(n_skills):
        idx = skill_ids_np == skill_id
        if not idx.any():
            continue
        plt.scatter(pca_result[idx, 0], pca_result[idx, 1], label=f'Skill {skill_id}', alpha=0.6)
    plt.legend()
    plt.title('PCA of Representation φ(s)')
    plt.savefig(f"{folder}/pca_representation_pca.png")
    plt.close()




def evaluate_skills(env_name, policy, representation, global_step, writer=None, n_skills=5, steps_per_skill=512, video_dir='skill_videos'):
    os.makedirs(video_dir, exist_ok=True)

    temp_buffer = ReplayBuffer(capacity=steps_per_skill * n_skills)
    temp_buffer.batch_size = 512

    # combined_heatmap = np.zeros_like(heatmap)

    for skill_id in range(n_skills): # For every z vector, I will perform ONLY the action related to that skill and get the reward
        env = gym.make(env_name, render_mode='rgb_array')
        obs, _ = env.reset()
        # obs = torch.tensor(obs, dtype=torch.float32, device=device).view(1, -1) ## NO need to flatten CNN encoder in representation function will handle it
        obs = torch.unsqueeze(torch.tensor(obs, dtype=torch.float32, device=device), 0).permute(0, 3, 1, 2)

        # USING ONE HOT ENCODED SKILLS HERE SO THAT WE CAN FIND THE DIFFERENT SKILLS SEPARATELY
        skill = torch.zeros(1, n_skills, device=device)
        skill[0, skill_id] = 1.0  # One-hot vector for evaluation


        frames = []
        total_intrinsic_reward = 0
        step_count = 0

        norms = []

        
        while step_count < steps_per_skill:
            with torch.no_grad():
                action = policy.get_action(obs, skill, eval=True).item()
            next_obs, _, terminated, truncated, _ = env.step(action)
            frame = env.render()
            frames.append(frame)

            # next_obs_tensor = torch.tensor(next_obs, dtype=torch.float32, device=device).view(1, -1)
            next_obs_tensor = torch.unsqueeze(torch.tensor(next_obs, dtype=torch.float32, device=device), 0).permute(0, 3, 1, 2) ## Encoding doesn't matter because get_latent_representation encodes
            delta_phi = representation.get_latent_representation(next_obs_tensor) - representation.get_latent_representation(obs)
            norms.append(delta_phi.norm(dim=1))  
            intrinsic_reward = torch.einsum('bi,bi->b', delta_phi, skill).item()
            temp_buffer.store(obs, skill, torch.tensor([action]), torch.tensor([intrinsic_reward]), next_obs_tensor, torch.tensor([~(terminated or truncated)]))
            total_intrinsic_reward += intrinsic_reward

            if terminated or truncated:
                break

            obs = next_obs_tensor
            step_count += 1

        env.close()
        
        heatmap_path = os.path.join(video_dir, f"skill_{skill_id}_coverage.png") 
        heatmap, coverage = compute_pacman_heatmap(frames)
        plt.imsave(heatmap_path, heatmap / heatmap.max(), cmap='hot')
        if writer:
            writer.add_scalar(f"eval/coverage_skill_{skill_id}", coverage, global_step)
        if skill_id == 0:
            combined_heatmap = (heatmap > 0).astype(np.float32)
        else:
            combined_heatmap += (heatmap > 0).astype(np.float32)

        video_path = os.path.join(video_dir, f"skill_{skill_id}.mp4")
        imageio.mimsave(video_path, frames, fps=30)

        # writer.add_scalar("representation/mean_delta_phi_norm", delta_phi.norm(dim=1).item(), global_step)
        mean_norm = torch.cat(norms).mean().item()
        writer.add_scalar("representation/mean_delta_phi_norm", mean_norm, global_step)
        # heatmap_path = os.path.join(video_dir, f"skill_{skill_id}_heatmap.png")
        # compute_pacman_heatmap(frames, heatmap_path)

        del frames
        torch.cuda.empty_cache()  
        # print(f"Skill {skill_id}: steps = {step_count}, intrinsic reward = {total_intrinsic_reward:.2f}, saved to {video_path}")
        if writer:
            writer.add_scalar(f"eval/intrinsic_reward_skill_{skill_id}", total_intrinsic_reward, global_step)
            writer.add_scalar(f"eval/episode_length_skill_{skill_id}", step_count, global_step)
    
    # visualize_representation(temp_buffer, representation, folder=video_dir, n_skills=n_skills)

    visualize_representation(temp_buffer, representation, global_step, folder=video_dir, n_skills=n_skills)
 
    ## Evaluate action distribution -- irrespective of the skill
    states, skills, _, _, _, _ = temp_buffer.sample()
    with torch.no_grad():
        dist  = policy.get_policy_distribution(states, skills)
        probs = dist.probs
    mean_probs = probs.mean(dim=0)

    for a in range(9): ## 9 actions in Ms. Pacman
        writer.add_scalar(f"action_dist/action_{a}", mean_probs[a].item())
    writer.add_histogram("action_dist/all_actions", mean_probs.cpu().numpy())


    # aggregate across all skills by poooling heatmap
    
    
    combined_coverage = compute_coverage(combined_heatmap)
    writer.add_scalar("eval/combined_coverage", combined_coverage, global_step)

In [ ]:


env = gym.make('ALE/MsPacman-v5')
# n_obs = env.observation_space.shape[0] * env.observation_space.shape[1] * env.observation_space.shape[2]
n_obs = 256 # Encoded dimension
n_actions = env.action_space.n # 9
n_skill = 10  # 0 - NOOP, 1 - UP, 2 - LEFT, 3 - RIGHT, 4 - DOWN

representation = RepresentationFunction(n_obs, n_skill) # phi
policy = SkillPolicy(n_obs, n_skill, n_actions, representation) # pi for dicrete SAC 


start_epoch = 0

# recovering from a failed run
chkpt_path = ''
if len(chkpt_path) > 0:
    if not os.path.exists(chkpt_path):
        raise ValueError("Load folder path doesn't exist")
    checkpoint = torch.load(chkpt_path)
    policy.policy.load_state_dict(checkpoint['policy_state_dict'])
    policy.q1_net.load_state_dict(checkpoint['q1_state_dict'])
    policy.q2_net.load_state_dict(checkpoint['q2_state_dict'])
    policy.q1_target_net.load_state_dict(checkpoint['q1_target_state_dict'])
    policy.q2_target_net.load_state_dict(checkpoint['q2_target_state_dict'])
    policy.policy_optimizer.load_state_dict(checkpoint['policy_optimizer_state_dict'])
    policy.q_optimizer.load_state_dict(checkpoint['q_optimizer_state_dict'])

    representation.representation_func.load_state_dict(checkpoint['representation_state_dict'])
    representation.optimizer.load_state_dict(checkpoint['representation_optimizer_state_dict'])
    representation.lambda_param = torch.tensor(checkpoint['lambda_param'], requires_grad=True)
    representation.lambda_optimizer.load_state_dict(checkpoint['lambda_optimizer_state_dict'])

    start_epoch = checkpoint['epoch'] + 1


# Training loop
num_epochs = 551

drl = DRL(buffer_size = 10000)

if not os.path.exists(f"./model/{timestamp}"):
    os.makedirs(f"./model/{timestamp}")

folder = f'./model/{timestamp}'

for epoch in range(start_epoch, num_epochs + start_epoch):
    print(f'EPOCH: {epoch}')
    drl.rollout(policy, epoch, n_skill, representation.encoder)
    
    representation.update(drl.replay_buffer, epoch) # update phi and lambda 
    policy.update(drl.replay_buffer, epoch) # update policy with scratch implemented discrete SAC 
    if epoch % 10 == 0:
        ## EVALUATING SKILL
        evaluate_skills('ALE/MsPacman-v5', policy, representation,\
                epoch, writer=writer, n_skills=n_skill, video_dir=f'{folder}/video_eval_{epoch}') ## ONLY doing for the first 3 skills

        ## EVALUATING REPRESENTATION FUNCTION
        # visualize_representation(replay_buffer=drl.replay_buffer,\
        #      representation=representation, folder=f"{folder}/video_eval_{epoch}") ## DOING THIS IN SkILL EVALUATION WHICH DOES ONE HOT ENCODED SKILLS
        # writer.add_image("eval/tSNE", \
        #      torch.tensor(np.array(Image.open(f"{folder}/video_eval_{epoch}/representation_tsne.png"))).permute(2, 0, 1), epoch)

        gc.collect()
    cleanup_memory()

torch.save({
    'epoch': num_epochs,
    'policy_state_dict': policy.policy.state_dict(),
    'q1_state_dict': policy.q1_net.state_dict(),
    'q2_state_dict': policy.q2_net.state_dict(),
    'q1_target_state_dict': policy.q1_target_net.state_dict(),
    'q2_target_state_dict': policy.q2_target_net.state_dict(),
    'policy_optimizer_state_dict': policy.policy_optimizer.state_dict(),
    'q_optimizer_state_dict': policy.q_optimizer.state_dict(),

    'representation_state_dict': representation.representation_func.state_dict(),
    'representation_optimizer_state_dict': representation.optimizer.state_dict(),
    'lambda_param': representation.lambda_param.detach().item(),
    'lambda_optimizer_state_dict': representation.lambda_optimizer.state_dict(),
}, f"./model/{timestamp}/checkpoint.pth")

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
/project/mlobo6/miniconda3/envs/drl/lib/python3.10/site-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return disable_fn(*args, **kwargs)


EPOCH: 0


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 210) to (160, 224) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

EPOCH: 1
EPOCH: 2
EPOCH: 3
EPOCH: 4
EPOCH: 5
EPOCH: 6


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 2 has a total capacity of 44.45 GiB of which 50.00 MiB is free. Including non-PyTorch memory, this process has 44.39 GiB memory in use. Of the allocated memory 38.72 GiB is allocated by PyTorch, and 5.35 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)